# Chapter10. Sequence hacking, hash, slice
- <h5>다음과 같은 기능을 지원하는 Vector class를 구현</h5>
    - Sequence Protocol: len과 getitem method
    - 여러 항목을 가진 객체를 안전하게 표현
    - slicing을 지원해서 새로운 vector object 생성
    - 포함된 component 값을 모두 고려한 set hashing
    - customize한 format언어 확장
    - read-only property였던 것을 교체하기 위해 \__getattr__() method로 동적 속성 접근을 구현
- <h5>protocol과 duck-typing</h5>


## 10.2 Vector_v1: Vector2d의 호환
- <h5>seq constructor(생성자)는 내장 seq처럼 반복형을 param으로 받게 하는 것이 좋음</h5>

In [1]:
from array import array
import reprlib
import math
import collections

In [2]:
class Vector_v0:
    typecode='d'

    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components) # array('d', [0, 1, 2, ...])처럼 표현하기 위한 reprlib.repr
        components = components[components.find('['):-1] # 'array(' 와 'd'를 제거하여 표현하기 위함
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x**2 for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, idx):
        return self._components[idx]
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [3]:
print(repr(Vector_v0([3.1, 4.2])))
print(repr(Vector_v0((3,4,5))))
print(repr(Vector_v0(range(10))))

Vector([3.1, 4.2])
Vector([3.0, 4.0, 5.0])
Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])


## 10.3 Protocol and duck-typing
- <h5>OOP에서 protocol은 문서에만 정의되어 있고, 실제 code에서는 정의되지 않는 비공식 interface</h5>
    - 예를 들어, python의 sequence protocol은 len()과 getitem() method를 동반할 뿐
    - 해당 클래스의 superclass가 무엇인지는 중요하지 않음. 중요한 method만 제공하면 됨
- <h5>아래 예제에서 sequence protocol을 구현하므로 python에서 제공하는 여러 기능을 활용할 수 있음</h5>
    - 코드내에서 sequence protocol을 따른다고 정의된 곳은 없음
    - 해당 code는 sequence처럼 '동작'
- <h5>protocol은 비공식이며, 강제로 적용되는 사항이 아님. 즉, 일부만 구현할 수도 있음</h5>
    - 반복 기능 지원하려면 \__getitem__() method만 구현해도 됨

In [4]:
import collections
Card = collections.namedtuple('Card', ['rank', 'suit'])

## __len__과 __getitem__method가 구현 --> sequence protocol
class FrenchDeck:
    rank = [str(n) for n in range(2,11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
    
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

## 10.4 Vector ver.2: Slice가능한 sequence
- <h5>10.3의 FrenchDeck 예제에서 len, getitem을 method를 추가하였기 때문에, slicing이 가능해짐</h5>
    - Vector class도 똑같이 해보자
- <h5>근데, 일반적인 slicing만 해서는 다음과 같이 array형태로 return되어 버림</h5>

In [5]:
v0 = Vector_v0([x for x in range(0,7)])
print(v0[2:5])

array('d', [2.0, 3.0, 4.0])


In [6]:
import numbers

class Vector_v1:
    typecode='d'

    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components) # array('d', [0, 1, 2, ...])처럼 표현하기 위한 reprlib.repr
        components = components[components.find('['):-1] # 'array(' 와 'd'를 제거하여 표현하기 위함
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x**2 for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, idx):
        cls = type(self)
        # idx인수가 slice면, _components array의 slice에서 Vector 생성자를 사용, object를 생성
        if isinstance(idx, slice):
            return cls(self._components[idx]) 
        # idx인수가 정수형이면, _components에서 해당 항목을 가져와서 반환함
        elif isinstance(idx, numbers.Integral):
            return self._components[idx]
        else:
            msg ='{} indices must be intergers'.format(cls.__name__)
            raise TypeError(msg.format(cls=cls))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [7]:
v0 = Vector_v0([x for x in range(0,7)])
print(v0[2:5])
v1 = Vector_v1([x for x in range(0,7)])
print(v1[2:5])

array('d', [2.0, 3.0, 4.0])
(2.0, 3.0, 4.0)


### 10.4.1 Slicing의 작동 방식
- <h5>slice는 내장 자료형이다.</h5>
- <h5>S.indices(len) --> (start, stop, stride)</h5>
    - 길이가 len인 sequence S가 나타내는 확장된 slice의 start와 stop index 및 stride 길이를 계산
    - 경계를 벗어난 index는 일반적인 slice 처리 방법과 동일하게 잘라냄
    - 즉, 경계안에 들어가도록 조정된 0이나 양수인 start, stop, stride로 구성된 '정규화된' tuple 생성

In [8]:
#print(dir(slice))
class MySeq:
    def __getitem__(self, idx):
        return idx
    
s = MySeq()
print('s[1:4:2]                                   : ', s[1:4:2])
print('s[1:4:2, 9]                               : ', s[1:4:2, 9])
print('s[1:4:2, 7:9]                            : ', s[1:4:2, 7:9])
print('slice(None, 10, 2).indices(5)     : ', slice(None, 10, 2).indices(5))
print('slice(-3, None, None).indices(5): ', slice(-3, None, None).indices(5))

s[1:4:2]                                   :  slice(1, 4, 2)
s[1:4:2, 9]                               :  (slice(1, 4, 2), 9)
s[1:4:2, 7:9]                            :  (slice(1, 4, 2), slice(7, 9, None))
slice(None, 10, 2).indices(5)     :  (0, 5, 2)
slice(-3, None, None).indices(5):  (2, 5, 1)


### 10.4.2 slice를 인식하는 \__getitem__()

In [9]:
v7 = Vector_v1(range(7))
print(v7[-1])
print(v7[1:4])
print(v7[-1:])
# 다차원 indexing을 지원 안함, index나 slice로 구성된 tuple은 에러 발생 
print(v7[1,2]) 

6.0
(1.0, 2.0, 3.0)
(6.0,)


TypeError: Vector_v1 indices must be intergers

## 10.5 Vector ver.3: 동적 속성 접근(dynamic attribute access)


- 찾고 있는 attribute를 찾지 못하면, getattr() method를 호출함 --> 해당 object에 찾고 있는 attribute가 있는지 검사함
- 해당 속성이 없으면 object의 class에서 더 검색함
- 그래도 찾지 못할 경우, self와 attribute명을 문자열로 전달해서 object의 클래스에 정의된 getattr() method를 호출함
- Vector_v2의 \__getattr__() method는 찾고 있는 속성이 x, y, z, t중 하나인지 검사하고, 이 중 하나의 경우에는 해당 vector component를 return함